In [1]:
import torch
import sys

sys.path.append("./Models")
from FiniteHorizon import *
import math

In [2]:
t_dim = 192
r_dim = 96
z_dim = 10

B = 2
T = 5
f = 20

# Cylindrical coordinates r, theta, z
coor_ranges = [0.0, 0.0, -5.0, 48.0, 2*math.pi, 5.0]
voxel_sizes = [abs(coor_ranges[3] - coor_ranges[0]) / r_dim, 
              abs(coor_ranges[4] - coor_ranges[1]) / t_dim,
              abs(coor_ranges[5] - coor_ranges[2]) / 1] # since BEV

in_dim = 30 # Input feature size per point
enc_dim = 128 # Size of point pillars
z_latent_dim = 20 # Hidden state size for each cell
latent_dim = int(z_dim * z_latent_dim) # Size of output of motion net
out_dim = 26 # Number of semantic classes

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
model = FiniteHorizon(voxel_sizes, coor_ranges, device=device)

In [4]:
m = torch.distributions.multivariate_normal.MultivariateNormal(torch.zeros(in_dim), 10.0*torch.eye(in_dim))
pc_lists = []
for b_i in range(B):
    b_list = []
    for t_i in range(T):
        b_list.append(m.sample((10000, )).to(device))
    pc_lists.append(b_list)
preds = model(pc_lists)
print(preds.shape)

torch.Size([2, 96, 192, 10, 26])
